In [ ]:
import os
import pandas as pd
import pdfplumber
import requests
import json
import re
import warnings
warnings.filterwarnings("ignore")
from openai import OpenAI
from typing import List  
  
from tqdm import tqdm  
from langchain_core.prompts import ChatPromptTemplate  
from langchain_core.pydantic_v1 import BaseModel, Field  
from langchain_core.output_parsers import JsonOutputParser  
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import UnstructuredFileLoader  
from langchain_text_splitters import RecursiveCharacterTextSplitter

/Users/taijieshengwu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/taijieshengwu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/taijieshengwu/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or

In [69]:
# apikey = "sk-mVpKLjvvbUO1CerUNSIRkBKtYaRYdF9anaotBTc164Y6tS9x"
# apikey="sk-q8FFK5jSqdIgViWq4cKYPkltTkIT4f6sM2TE4oEOoFW3oHHL"
# apikey = "sk-slececspLTc947d0am0X3G0y5mwo7qVAofFGTsq3citFJeya"
# BAICHUAN API
apikey = "sk-25a10e97a9fbfed98a9c591bbc605792"
data_dir = '/Users/taijieshengwu/A-QA-System-for-Public-Art-Design/相关文本资料汇总'

In [62]:
# # 初始化一个空的 DataFrame
# pdf_data = []

# # 遍历目录下所有 PDF 文件
# for filename in os.listdir(data_dir):
#     if filename.endswith(".pdf"):
#         file_path = os.path.join(data_dir, filename)
#         try:
#             with pdfplumber.open(file_path) as pdf:
#                 text = ''
#                 filename = filename[:-4]
#                 for page in pdf.pages:
#                     text += page.extract_text() or ''
#                 if "_" in filename:
#                     title, author = filename.rsplit("_",1)
#                 else:
#                     title, author = filename, ""

#                 pdf_data.append({
#                     "title": title,
#                     "author": author,
#                     "content": text
#                 })
#         except Exception as e:
#             print(f"读取失败：{filename}, 错误：{e}")

# # 转换为 DataFrame
# df = pd.DataFrame(pdf_data)
# print(df['title'])
# df.to_csv("/Users/taijieshengwu/A-QA-System-for-Public-Art-Design/data.csv", index=False, encoding="utf-8-sig")
df = pd.read_csv("/Users/taijieshengwu/A-QA-System-for-Public-Art-Design/data.csv", encoding="utf-8-sig")

In [63]:
def clean_text_compact(text):
    # 去除 HTML 标签
    text = re.sub(r'<.*?>', '', text)
    # 去除所有空白字符（空格、换行、制表符等）
    text = re.sub(r'\s+', '', text)
    return text

def split_document(text):
    try:  
        cleaned_text = clean_text_compact(text)
        text_splitter = RecursiveCharacterTextSplitter(  
            chunk_size=1024,  
            chunk_overlap=128  
        )  
        documents = text_splitter.split_text(cleaned_text)
        return documents
    except:
        return text

# 示例调用
documents = split_document(df.iloc[0]['content'])
print(documents)
print("段落数：", len(documents))

['艺术教育115艺术管理共同体美学意识的构建———以剧院公共艺术教育策划与管理为例邓玉琪王小龙摘要艺术管理美学作为艺术美学与管理美学的交叉研究，是基于当代艺术主体关系转移、艺术过程要素转变、艺术内容多元转化的前提，在艺术管理视域下提出的美学观念，亦是对艺术美学传统观念的创造性转化与创新性发展。由于艺术管理介入艺术活动，艺术的美学意识发生了多维度的重构，如在主体意识、过程意识、内容意识等层面构建共同体意识。这种共同体意识的构建在剧院管理，尤其在剧院公共艺术教育策划与管理中十分必要。关键词艺术管理;艺术管理美学;剧院公共艺术教育;共同体意识剧院公共艺术教育策划与管理属于剧院管理的和内容，以及城市居民在剧院空间中彼此交流、对研究范畴，剧院管理是艺术管理的分支。从艺术门话的中介;剧院服务作为联结剧院与人、与社会的类上细分，艺术管理可分为音乐管理、戏剧管理、纽带，其公共服务是面向社会公众，是剧院艺术教舞蹈管理、电影管理等;从艺术机构上细分，艺术育活动区别于学校艺术教育的特色所在，并成为剧管理可分为演出现场管理、剧院管理、电影院管理院面向更多社会公众的渠道;剧院文化作为联结剧等。随着艺术管理学科的建设与完善，艺术管理的院与艺术、与人、与社会、与城市的纽带，以教育分类方式会更加丰富。共同体意识在不同领域具有为剧院文化输出的方式，让更多城市居民接受剧院不同的性质，在政治、经济领域属于哲学意识;在文化、体验剧院艺术，以此丰富精神生活。剧院空民族、国家层面属于文化意识;在艺术领域属于美间、剧院艺术、剧院服务、剧院文化是剧院公共艺术教育策划与管理的四大要素，都与人有关，人即学意识。艺术管理美学研究的是艺术管理者以主体主体，包括城市居民即剧院公共艺术教育的受众、身份介入艺术创作者、表演者、接受者的固有体策划与管理者、教育者，同时，剧院空间设计决定系，打破三角形结构，构建菱形结构以后，艺术的着公共艺术教育的实施环境(剧院)，剧院艺术本主体与客体构成多重审美关系、形式与内容构成多体决定着剧院公共教育的实施媒介(艺术)，剧院样审美关系、过程与结果构成多元审美关系，以及服务导向决定着剧院艺术教育的实施特性(公菱形结构中的审美要素等内容。共)，剧院文化理念决定着剧院公共艺术的实施方式(教育)。一、“主体”共同体意识的构建在剧院公共艺术教育策划与管理中，主体因处剧院作为一座城市的文化名片，肩负城市文

In [64]:
QA_PAIRS_SYSTEM_PROMPT = """  
<Context></Context> 你是一个面向公共艺术设计领域的问答系统，标记中是一段关于公共艺术的文献文本，学习和分析它，并整理学习成果：  
- 提出问题并给出每个问题的答案,里面不要出现意义不明的代词。  
- 答案需详细完整，尽可能保留原文描述。  
- 答案可以包含普通文字、链接、代码、表格、公示、媒体链接等 Markdown 元素。  
- 最多提出 30 个问题。  
"""

QA_PAIRS_HUMAN_PROMPT = """  
请按以下格式整理学习成果:  
<Context>  
文本  
</Context>  
[  
{{"question": "问题1","answer":"答案1"}},  
{{"question": "问题2","answer":"答案2"}},  
]  
------  
  
我们开始吧!  
  
<Context>  
{text}  
<Context/>  
"""

prompt = ChatPromptTemplate.from_messages([  
	("system", QA_PAIRS_SYSTEM_PROMPT),  
	("human", QA_PAIRS_HUMAN_PROMPT)  
])

In [75]:
# client = OpenAI(
#     # defaults to os.environ.get("OPENAI_API_KEY")
#     api_key=apikey,
#     base_url="https://api.chatanywhere.tech/v1"
# )

# baichuan API
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=apikey,
    base_url="https://api.baichuan.ai/v1"
)

In [ ]:
completion = client.chat.completions.create(
    model="Baichuan4",
    messages=[{
            "content": "今天天气怎么样？使用json格式输出",
            "role": "user"
      }
    ]
)
print(completion.choices[0].message)
              

In [71]:
class QaPair(BaseModel):  
	question: str = Field(description='问题内容')  
	answer: str = Field(description='问题的回答')  
	  
  
class QaPairs(BaseModel):  
	qas: List[QaPair] = Field(description='问答对列表')

parser = JsonOutputParser(pydantic_object=QaPairs)




In [79]:


# def chatanywhere_llm(input: dict):
#     messages = [
#         {"role": "system", "content": QA_PAIRS_SYSTEM_PROMPT},
#         {"role": "user", "content": QA_PAIRS_HUMAN_PROMPT.format(text=input['text'])},
#     ]
#     response = client.chat.completions.create(
#         model="Baichuan4-Air", # 
#         messages=messages,
#         temperature=0.7
#     )
#     return response.choices[0].message.content
BASE_URL = "https://api.baichuan-ai.com/v1/chat/completions"
def chatanywhere_llm(input: dict):
    messages = [
        {"role": "system", "content": QA_PAIRS_SYSTEM_PROMPT},
        {"role": "user", "content": QA_PAIRS_HUMAN_PROMPT.format(text=input['text'])}
    ]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {apikey}"
    }

    payload = {
        "model": "Baichuan2-Turbo",
        "messages": messages,
        "temperature": 0.7,
    }

    response = requests.post(BASE_URL, headers=headers, json=payload, timeout=60)
    if response.status_code != 200:
        raise RuntimeError(f"调用失败：{response.status_code}\n{response.text}")
    return response.json()['choices'][0]['message']['content']


# 包成 Runnable
llm = RunnableLambda(chatanywhere_llm)
parser = JsonOutputParser(pydantic_object=QaPairs)
chain = llm | parser

In [88]:
json_list = []       
non_json_list = []   
print(len(df))

for i in range(200,204): #
    content = df.iloc[i]['content']
    documents = split_document(content)
    if not isinstance(documents, (list, tuple)):
        print(f"Skipping row {i}: documents is not list-like, got {type(documents)}")
        continue

    bar = tqdm(total=len(documents), desc=f"Processing row {i}")

    for idx, doc in enumerate(documents):
        bar.update(1)
        try:
            out = chain.invoke({'text': doc})
            for qa in out:  # ✅ 因为 out 就是 list，不需要 .qas
                try:
                    qa_dict = qa.dict() if hasattr(qa, 'dict') else qa
                    json.dumps(qa_dict, ensure_ascii=False)
                    json_list.append(qa_dict)
                    print(qa_dict)

                    with open('dataset.json', 'a', encoding='utf-8') as f:
                        json.dump(qa_dict, f, ensure_ascii=False)
                        f.write('\n')

                except Exception as e:
                    non_json_list.append(str(qa))
                    with open('non_json_outputs.txt', 'a', encoding='utf-8') as f:
                        f.write(str(qa) + '\n')

        except Exception as e:
            print(f"Error invoking chain at row {i}, doc {idx}: {e}")
            non_json_list.append(doc)
            with open('non_json_outputs.txt', 'a', encoding='utf-8') as f:
                f.write(doc + '\n')

print(f"总共收集到合法QA数：{len(json_list)}")


249


Processing row 200:  67%|██████▋   | 2/3 [10:01<05:00, 300.67s/it]


Processing row 200:  67%|██████▋   | 2/3 [00:12<00:06,  6.29s/it]

{'question': '什么是公共艺术?', 'answer': '公共艺术是在现代社会对“公共性”的要求下，通过艺术招标、竞标、创作、监督、质询、评审等机制社会化程序进行的，但这些程序往往不规范与不完善。'}
{'question': '公共艺术遇到什么困难和混乱?', 'answer': '公共艺术在现代社会遇到了很多困难和混乱，其中包括社会化程序的不规范与不完善，以及公共性的难度。'}
{'question': '如何解决公共艺术的难题?', 'answer': '解决公共艺术的难题需要从现代性的高度思考和建构公共艺术的评价标准和运作机制。'}
{'question': '公共性是什么意思?', 'answer': '公共性意味着交往，意味着沟通，它在尊重每一个社会个体平等权利的基础上，强调共同的社会秩序和个人的社会责任。'}
{'question': '公共艺术是否能从公共利益、立场出发，体现公众趣味与意愿?', 'answer': '这是一个理论上的问题，实际上很难做到。但是，我们应该积极主动地思考和解决这个问题。'}
{'question': '公共艺术的发展是否受到权力的影响?', 'answer': '是的，公共艺术始终紧紧攀附在社会制度、文化体制与权力系统上。'}
{'question': '公共艺术是否能自律发展?', 'answer': '这是一个棘手的问题，需要我们从现代性的立场去看待。'}
{'question': '公共艺术是否能在中国当下社会中实现?', 'answer': '虽然公共艺术无法全面、彻底地在中国当下社会中实现，但我们应该积极主动地思考和解决这个问题。'}
{'question': "公共艺术是否涉及到'何谓公共性'这一现代性问题?", 'answer': "是的，公共艺术涉及到'何谓公共性'这一现代性问题。"}
{'question': '公共艺术是否能比过去做得更好?', 'answer': "是的，我们可以从现代性的立场，把当代公共艺术视为建构'公共领域'不可分割的一部分，从而比过去做得更好。"}


Processing row 200: 100%|██████████| 3/3 [00:25<00:00,  8.98s/it]

...

{'question': '公共艺术专业的课程如何引导学生艺术创作？', 'answer': '公共艺术专业的课程通常会通过理论讲述和与专业教师的交流两种方法来引导学生艺术创作。课堂中的理论内容会转化为创作实践，帮助学生建立起自己的艺术观念。'}
{'question': '艺术观念与艺术风格之间的关系是什么？', 'answer': '艺术观念与艺术风格之间的关系可以理解为动机与表达的关系。艺术风格是艺术家用来表达其艺术观念的一种方式。'}
{'question': '如何通过风格培养艺术观念？', 'answer': '通过风格培养艺术观念的方法是，从创作成果所包含的信息反向解剖艺术思维，以创作动机去理解艺术观念，进而对自身创作过程进行反思并触发创作思维的形成。'}
{'question': '艺术观念是如何形成的？', 'answer': '艺术观念的形成并非偶然，而是受到公众的审美思想和习惯的影响，同时也遵循着自身的发展规律。'}
{'question': '为什么培养学生的艺术观念很重要？', 'answer': '培养学生的艺术观念非常重要，因为如果学生只重视技巧而忽视观念，他们的创作实践就会处于被动状态，难以达到高质量的专业人才培养。'}
{'question': '汕头大学在公共艺术课程体系中是如何设置课程的？', 'answer': '汕头大学在公共艺术课程体系中设置了包括公共观念与艺术观念、造型/空间、现代造型语言、立体与空间工作坊、空间营造计划、艺术工程综合基础、文案写作提案方法等必修课程。'}
{'question': '汕头大学公共艺术专业的课程设计分为几个阶段？', 'answer': '汕头大学公共艺术专业的课程设计分为两个阶段：第一阶段为理论学习，第二阶段为创作实践。'}
{'question': '在课程设计中，如何平衡技巧与观念的教学？', 'answer': '在课程设计中，可以通过将平面设计和立体造型作为创作基础，同时以风格作为视觉表达方向，来平衡技巧与观念的教学。'}
{'question': '如何评价当前应试教育下艺术类专业招生的情况？', 'answer': '当前应试教育下艺术类专业招生为高校输送了大量生源，但也造成了学生重技巧而轻观念的审美能力缺陷。'}
{'question': '如何解决学生在艺术观念上的短板？

{'question': '课程设计的主要目标是什么？', 'answer': '课程设计的主要目标是建立学生关于艺术活动中“艺术风格”的理论性认识，以及掌握公共艺术创作中的“风格”表现。'}
{'question': '课程设计分为哪两个阶段？', 'answer': '课程设计分为创作基础阶段和创作训练阶段。'}
{'question': '创作基础阶段主要关注什么？', 'answer': '创作基础阶段主要关注建立学生的公共艺术创作“观念”，掌握公共艺术创作中的“风格”表现。'}
{'question': '创作训练阶段的教学内容是什么？', 'answer': '创作训练阶段的教学内容以学生分组创作为主，通过“创作构思—资料收集—研究作品形式和风格—寻求表现方法—作品制作”的创作流程，最终完成作品。'}
{'question': '学生在创作训练阶段需要做什么？', 'answer': '学生在创作训练阶段需要选择某种“风格”进行构思，整理构成元素后和作业模型。'}
{'question': '课程设计中提到的“风格”是什么？', 'answer': '课程设计中提到的“风格”是一个艺术的和美学的范畴，在艺术创作中既是艺术家的标签，也是一件作品和观众对话的重要切入点。'}
{'question': '学生如何理解“风格”的构成元素和方式？', 'answer': '学生通过对作品的构成元素进行分析总结，通过创作创意表述方案展示，以及作品的完整制作来理解和掌握“风格”的构成元素和方式。'}
{'question': '课程设计中提到的艺术风格有哪些例子？', 'answer': '课程设计中提到的艺术风格有构成主义风格和波普艺术风格。'}
{'question': '学生如何解决“风格”和“主义”的概念差异？', 'answer': '学生需要梳理艺术发展过程中所呈现的风格特征，同时打破艺术史理论的线性叙述来归纳艺术风格。'}


{'question': "在艺术创作中，'风格'对作品有何影响？", 'answer': "在艺术创作中，'风格'是艺术家表达其独特视角和审美观的重要手段。作品风格不仅能够吸引观众的注意力，还能够在视觉上形成强烈的识别度，使作品在众多艺术作品中脱颖而出。同时，风格也是艺术家与观众之间沟通的一种方式，通过风格传达出艺术家的思想、情感和意图。此外，风格还能够反映出艺术家的文化背景、历史时期和社会环境等因素，从而为作品增添更多的文化内涵和历史价值。总之，风格是艺术创作中不可或缺的一部分，它能够丰富作品的表现力和感染力，让观众在欣赏作品的过程中获得更加深刻的感受和体验。"}
{'question': "如何在教学中引导学生理解'风格'的概念？", 'answer': "在教学中，可以通过以下几个步骤来引导学生理解'风格'的概念：首先，让学生观察和分析不同类型的艺术作品，从中找出相似的风格特征；其次，让学生尝试用自己的语言描述这些风格特征，并探讨它们是如何影响作品的整体效果的；接着，可以让学生尝试模仿某种风格进行创作，以此来加深对风格的理解和掌握；最后，鼓励学生在创作中尝试融合多种风格元素，创造出属于自己的独特风格。通过这样的教学过程，学生不仅能够理解'风格'的概念，还能够学会如何运用风格来表达自己的创意和个性。"}
{'question': "在公共艺术领域，作品风格与'场所精神'之间的关系是什么？", 'answer': "在公共艺术领域，作品风格与'场所精神'之间的关系密切。'场所精神'是指一个地方特有的氛围和特质，它受到地理位置、历史文化、社会环境等多种因素的影响。而作品风格则是艺术家根据'场所精神'的特点和需求，选择适合的表现手法和艺术语言，使作品与环境相协调，增强作品的地域特色和文化内涵。因此，作品风格是艺术家对'场所精神'的一种回应和诠释，它能够使作品更好地融入环境，与周围的空间和景观形成和谐的关系，提升公共空间的艺术品质和文化价值。"}
{'question': '如何利用课题讨论的方式引导学生分析不同风格的艺术现象？', 'answer': '利用课题讨论的方式引导学生分析不同风格的艺术现象，可以通过以下步骤进行：首先，确定一个具体的课题，如“各类写实风格比较”；然后，让学生分组搜集相关资料，包括不同艺术家的作品、历史背景、文化影响等；接着，在课堂上进行小

{'question': '《全球变暖》海报和雕塑作品的作者是谁？', 'answer': '《全球变暖》海报和雕塑作品的作者是张恩闰、陈俊杰、姚丽琳。'}
{'question': '《全球变暖》雕塑作品中的“456的解剖模型”有什么特点？', 'answer': '《全球变暖》雕塑作品中的“456的解剖模型”外壳打开后可以看见“内脏”，涂鸦作者借鉴了英国艺术家达明·赫斯特的艺术和冷峻的视觉特点。作者借鉴了这一类型的风格，式的细节看出作者的创作意图并非“科学再现”，观念和表现手法，寻求科学和艺术之间的关联性，通过夸张和比喻的表现手法传达出作者对于环境问题的批判和关注。'}
{'question': '《全球变暖》作品如何体现作者对于环境的关注？', 'answer': '《全球变暖》作品中白色液状物体象征地球两极的冰川，通过生物融化的状态传达给观众多层次的意涵，例如气候变暖所带来的冰川融化、环境污染对于生物生存状态的腐蚀等。'}
{'question': '课程如何通过研究“风格”来培养学生的艺术观念？', 'answer': '课程在教学设计上采取研究和实践相结合的方式，由艺术理论推进到作品创作，在“艺术观念”和作品的风格设定上，培养学生艺术创作中所需要的思维能力，目的在于掌握细节特征，达到创作所需要的严谨处理，从而在创作中熟练运用“风格化”的形式语言。'}
{'question': '资料的收集和整理在课程中是如何体现的？', 'answer': '资料的收集和整理是不断产生知识需求和解决问题的过程，主要体现在三个方面：一、收集和整理昆虫资料，例如外形特点、生活习性等，达到创作所需要的严谨处理；二、收集科普类的插图和模型，这类资料涉及作品的视觉特征，是作品风格的重要元素；三、学习和整理研究方式，首先需要了解同类型作品的表达方式和表现手段，在构思、色彩、造型等风格方面进行总结和研究。'}


{'question': '问题1', 'answer': '答案1'}
{'question': '问题2', 'answer': '答案2'}
{'question': '模型雕塑通常使用什么材料？', 'answer': '模型雕塑通常使用树脂在昆虫模型上表达观念象学。'}
{'question': '艺术家如何对模型进行处理以达到所需效果？', 'answer': '艺术家会翻制出自己需要的部分，然后进行组合或分解，例如剖开外壳和内部肌肉组织填充，模仿出“教内容”的形态。'}
{'question': '现代艺术家关注的主要环境问题是什么？', 'answer': '现代艺术家关注的主要环境问题是环境保护，尤其是全球变暖的问题。'}
{'question': '艺术家班克斯的作品《我不相信》是什么类型的绘画？', 'answer': '艺术家班克斯的作品《我不相信》是涂鸦绘画。'}
{'question': '洛伦佐·奎因在威尼斯双年展中的作品是什么？', 'answer': '洛伦佐·奎在威尼斯双年展中的作品是《扶墙的大手》。'}
总共收集到合法QA数：137
